In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1233917,2021-05-01,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1233918,2021-05-02,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1233919,2021-05-03,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1233920,2021-05-04,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
32943,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32945,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32947,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32949,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32951,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
759186,2021-05-01,Arkansas,Arkansas,5001,2095,36.00,5000,Arkansas,AR,Arkansas,State,3017804
759188,2021-05-02,Arkansas,Arkansas,5001,2095,36.00,5000,Arkansas,AR,Arkansas,State,3017804
759190,2021-05-03,Arkansas,Arkansas,5001,2095,36.00,5000,Arkansas,AR,Arkansas,State,3017804
759192,2021-05-04,Arkansas,Arkansas,5001,2096,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1233917,2021-05-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1233918,2021-05-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1233919,2021-05-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1233920,2021-05-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-05-05') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
470,2021-05-05,Snohomish,Washington,36622,569.00,53061,WA,County,822083,4454.78,69.21
938,2021-05-05,Cook,Illinois,539005,10563.00,17031,IL,County,5150233,10465.64,205.10
1405,2021-05-05,Orange,California,270507,4979.00,6059,CA,County,3175692,8518.05,156.78
1871,2021-05-05,Maricopa,Arizona,539632,9889.00,4013,AZ,County,4485414,12030.82,220.47
2337,2021-05-05,Los Angeles,California,1235037,23957.00,6037,CA,County,10039107,12302.26,238.64
...,...,...,...,...,...,...,...,...,...,...,...
1233225,2021-05-05,Wheeler,Oregon,26,1.00,41069,OR,County,1332,1951.95,75.08
1233430,2021-05-05,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1233604,2021-05-05,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1233774,2021-05-05,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
470,2021-05-05,Snohomish,Washington,36622,569.00,53061,WA,County,822083,4454.78,69.21,69.21,4454.78
938,2021-05-05,Cook,Illinois,539005,10563.00,17031,IL,County,5150233,10465.64,205.10,205.10,10465.64
1405,2021-05-05,Orange,California,270507,4979.00,6059,CA,County,3175692,8518.05,156.78,156.78,8518.05
1871,2021-05-05,Maricopa,Arizona,539632,9889.00,4013,AZ,County,4485414,12030.82,220.47,220.47,12030.82
2337,2021-05-05,Los Angeles,California,1235037,23957.00,6037,CA,County,10039107,12302.26,238.64,238.64,12302.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233225,2021-05-05,Wheeler,Oregon,26,1.00,41069,OR,County,1332,1951.95,75.08,75.08,1951.95
1233430,2021-05-05,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1233604,2021-05-05,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1233774,2021-05-05,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
793849,2021-05-05,Hale,Alabama,2222,76.00,1065,AL,County,14651,15166.20,518.74,518.74,15166.20,1
864119,2021-05-05,Clarke,Alabama,3506,61.00,1025,AL,County,23622,14842.10,258.23,258.23,14842.10,2
621081,2021-05-05,Lowndes,Alabama,1385,53.00,1085,AL,County,9726,14240.18,544.93,544.93,14240.18,3
462066,2021-05-05,Franklin,Alabama,4247,82.00,1059,AL,County,31362,13541.87,261.46,261.46,13541.87,4
550676,2021-05-05,Etowah,Alabama,13806,352.00,1055,AL,County,102268,13499.82,344.19,344.19,13499.82,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169529,2021-05-05,Platte,Wyoming,623,11.00,56031,WY,County,8393,7422.85,131.06,131.06,7422.85,19
757581,2021-05-05,Converse,Wyoming,999,17.00,56009,WY,County,13822,7227.61,122.99,122.99,7227.61,20
941926,2021-05-05,Lincoln,Wyoming,1401,12.00,56023,WY,County,19830,7065.05,60.51,60.51,7065.05,21
1048392,2021-05-05,Niobrara,Wyoming,159,2.00,56027,WY,County,2356,6748.73,84.89,84.89,6748.73,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
621081,2021-05-05,Lowndes,Alabama,1385,53.00,1085,AL,County,9726,14240.18,544.93,544.93,14240.18,3,1
793849,2021-05-05,Hale,Alabama,2222,76.00,1065,AL,County,14651,15166.20,518.74,518.74,15166.20,1,2
264095,2021-05-05,Walker,Alabama,7214,277.00,1127,AL,County,63521,11356.87,436.08,436.08,11356.87,28,3
549048,2021-05-05,Clay,Alabama,1551,56.00,1027,AL,County,13235,11718.93,423.12,423.12,11718.93,20,4
669798,2021-05-05,Greene,Alabama,923,34.00,1063,AL,County,8111,11379.61,419.18,419.18,11379.61,27,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668177,2021-05-05,Hot Springs,Wyoming,369,3.00,56017,WY,County,4413,8361.66,67.98,67.98,8361.66,17,19
918444,2021-05-05,Uinta,Wyoming,2209,13.00,56041,WY,County,20226,10921.59,64.27,64.27,10921.59,5,20
941926,2021-05-05,Lincoln,Wyoming,1401,12.00,56023,WY,County,19830,7065.05,60.51,60.51,7065.05,21,21
262856,2021-05-05,Teton,Wyoming,3744,9.00,56039,WY,County,23464,15956.36,38.36,38.36,15956.36,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.21,4454.78,19,23
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.21,4454.78,19,23
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.21,4454.78,19,23
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.21,4454.78,19,23
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.21,4454.78,19,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231026,2021-05-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1231027,2021-05-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1231028,2021-05-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1231029,2021-05-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
790557,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,518.74,15166.20,2,1,1.00
790558,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,518.74,15166.20,2,1,0.00
790559,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,518.74,15166.20,2,1,0.00
790560,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,518.74,15166.20,2,1,0.00
790561,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,518.74,15166.20,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957690,2021-05-01,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,0.00
957691,2021-05-02,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,0.00
957692,2021-05-03,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,0.00
957693,2021-05-04,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
618593,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14240.18,1,3,1.00,0.00
618594,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14240.18,1,3,0.00,0.00
618595,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14240.18,1,3,0.00,0.00
618596,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14240.18,1,3,0.00,0.00
618597,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14240.18,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616558,2021-05-01,Albany,Wyoming,4242,11.00,56001,WY,County,38880,10910.49,28.29,30.86,11021.09,23,4,0.00,0.00
616559,2021-05-02,Albany,Wyoming,4242,11.00,56001,WY,County,38880,10910.49,28.29,30.86,11021.09,23,4,0.00,0.00
616560,2021-05-03,Albany,Wyoming,4266,11.00,56001,WY,County,38880,10972.22,28.29,30.86,11021.09,23,4,24.00,0.00
616561,2021-05-04,Albany,Wyoming,4271,12.00,56001,WY,County,38880,10985.08,30.86,30.86,11021.09,23,4,5.00,1.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-05-05') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
315275,2021-05-05,Imperial,California,28280,721.00,6025,CA,County,181215,15605.77,397.87,397.87,15605.77,1,2,0.00,0.00
2337,2021-05-05,Los Angeles,California,1235037,23957.00,6037,CA,County,10039107,12302.26,238.64,238.64,12302.26,2,5,172.00,15.00
156967,2021-05-05,San Bernardino,California,296257,4668.00,6071,CA,County,2180085,13589.24,214.12,214.12,13589.24,3,4,69.00,9.00
552256,2021-05-05,Inyo,California,1424,38.00,6027,CA,County,18039,7894.01,210.65,210.65,7894.01,4,24,2.00,0.00
70181,2021-05-05,Stanislaus,California,61681,1050.00,6099,CA,County,550660,11201.29,190.68,190.68,11201.29,5,9,75.00,3.00
32517,2021-05-05,Riverside,California,298872,4576.00,6065,CA,County,2470546,12097.41,185.22,185.22,12097.41,6,6,95.00,2.00
55835,2021-05-05,San Joaquin,California,72784,1384.00,6077,CA,County,762148,9549.85,181.59,181.59,9549.85,7,18,60.00,2.00
87018,2021-05-05,Tulare,California,49514,837.00,6107,CA,County,466195,10620.88,179.54,179.54,10620.88,8,10,18.00,0.00
31667,2021-05-05,Fresno,California,101451,1677.00,6019,CA,County,999101,10154.23,167.85,167.85,10154.23,9,14,46.00,-1.00
500457,2021-05-05,Merced,California,31701,454.00,6047,CA,County,277680,11416.38,163.50,163.50,11416.38,10,8,19.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1169080,2021-05-05,Lassen,California,5693,24.00,6035,CA,County,30573,18621.01,78.50,78.50,18621.01,34,1,1.00,0.00
315275,2021-05-05,Imperial,California,28280,721.00,6025,CA,County,181215,15605.77,397.87,397.87,15605.77,1,2,0.00,0.00
669740,2021-05-05,Kings,California,22933,246.00,6031,CA,County,152940,14994.77,160.85,160.85,14994.77,11,3,22.00,0.00
156967,2021-05-05,San Bernardino,California,296257,4668.00,6071,CA,County,2180085,13589.24,214.12,214.12,13589.24,3,4,69.00,9.00
2337,2021-05-05,Los Angeles,California,1235037,23957.00,6037,CA,County,10039107,12302.26,238.64,238.64,12302.26,2,5,172.00,15.00
32517,2021-05-05,Riverside,California,298872,4576.00,6065,CA,County,2470546,12097.41,185.22,185.22,12097.41,6,6,95.00,2.00
197361,2021-05-05,Kern,California,108822,1361.00,6029,CA,County,900202,12088.62,151.19,151.19,12088.62,14,7,103.00,3.00
500457,2021-05-05,Merced,California,31701,454.00,6047,CA,County,277680,11416.38,163.50,163.50,11416.38,10,8,19.00,0.00
70181,2021-05-05,Stanislaus,California,61681,1050.00,6099,CA,County,550660,11201.29,190.68,190.68,11201.29,5,9,75.00,3.00
87018,2021-05-05,Tulare,California,49514,837.00,6107,CA,County,466195,10620.88,179.54,179.54,10620.88,8,10,18.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
315275,2021-05-05,Imperial,California,28280,721.00,6025,CA,County,181215,15605.77,397.87,397.87,15605.77,1,2,0.00,0.00
2337,2021-05-05,Los Angeles,California,1235037,23957.00,6037,CA,County,10039107,12302.26,238.64,238.64,12302.26,2,5,172.00,15.00
156967,2021-05-05,San Bernardino,California,296257,4668.00,6071,CA,County,2180085,13589.24,214.12,214.12,13589.24,3,4,69.00,9.00
552256,2021-05-05,Inyo,California,1424,38.00,6027,CA,County,18039,7894.01,210.65,210.65,7894.01,4,24,2.00,0.00
70181,2021-05-05,Stanislaus,California,61681,1050.00,6099,CA,County,550660,11201.29,190.68,190.68,11201.29,5,9,75.00,3.00
32517,2021-05-05,Riverside,California,298872,4576.00,6065,CA,County,2470546,12097.41,185.22,185.22,12097.41,6,6,95.00,2.00
55835,2021-05-05,San Joaquin,California,72784,1384.00,6077,CA,County,762148,9549.85,181.59,181.59,9549.85,7,18,60.00,2.00
87018,2021-05-05,Tulare,California,49514,837.00,6107,CA,County,466195,10620.88,179.54,179.54,10620.88,8,10,18.00,0.00
31667,2021-05-05,Fresno,California,101451,1677.00,6019,CA,County,999101,10154.23,167.85,167.85,10154.23,9,14,46.00,-1.00
500457,2021-05-05,Merced,California,31701,454.00,6047,CA,County,277680,11416.38,163.50,163.50,11416.38,10,8,19.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,397.87,15605.77,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,397.87,15605.77,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,397.87,15605.77,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,397.87,15605.77,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,397.87,15605.77,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4132,05/01/21,Lassen,5692,24.00,18617.73,78.50,78.50,18621.01,34,1,0.00,0.00
4133,05/02/21,Lassen,5692,24.00,18617.73,78.50,78.50,18621.01,34,1,0.00,0.00
4134,05/03/21,Lassen,5692,24.00,18617.73,78.50,78.50,18621.01,34,1,0.00,0.00
4135,05/04/21,Lassen,5692,24.00,18617.73,78.50,78.50,18621.01,34,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,397.87,15605.77,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,397.87,15605.77,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,397.87,15605.77,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,397.87,15605.77,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,397.87,15605.77,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4218,05/01/21,Merced,31594,454.00,11377.85,163.50,163.50,11416.38,10,8,0.00,0.00
4219,05/02/21,Merced,31594,454.00,11377.85,163.50,163.50,11416.38,10,8,0.00,0.00
4220,05/03/21,Merced,31657,454.00,11400.53,163.50,163.50,11416.38,10,8,63.00,0.00
4221,05/04/21,Merced,31682,454.00,11409.54,163.50,163.50,11416.38,10,8,25.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)